<a href="https://colab.research.google.com/github/AIAlchemy1/Generative-AI/blob/main/05_Retrieval_Augmented_Generation_RAG/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Setup
1) Clone https://github.com/plaggy/rag-gradio-sample-project and set up an environment with gradio_app/requirements.txt.

There you'll find the following files:
- [prep_scripts/markdown_to_text.py](https://github.com/plaggy/rag-gradio-sample-project/blob/main/prep_scripts/markdown_to_text.py) processes markdown into text; you won't need to change it.
- [prep_scripts/lancedb_setup.py](https://github.com/plaggy/rag-gradio-sample-project/blob/main/prep_scripts/lancedb_setup.py) is the file where the database is created and, in particular, an embedding model is defined.
- [gradio_app/backend/query_llm.py](https://github.com/plaggy/rag-gradio-sample-project/blob/main/gradio_app/backend/query_llm.py) defines what LLM is used.
- [gradio_app/app.py](https://github.com/plaggy/rag-gradio-sample-project/blob/main/gradio_app/app.py) creates the gradio app.

In this task you'll try not only OpenAI models, but also open-source models from Hugging Face Hub through InferenceClient interface (see [gradio_app/backend/query_llm.py](https://github.com/plaggy/rag-gradio-sample-project/blob/main/gradio_app/backend/query_llm.py)). Please don't forget to obtain a Hugging Face token for that (see here https://huggingface.co/settings/tokens).


A convenient way to work through the project is to test locally and keep committing the changes to the [HF Spaces](https://huggingface.co/spaces) repo. A space gets automatically rebuilt after each commit and you get a new version of your application up and running.

2) Create a new space with Gradio SDK. You'll get an almost empty repo, the only thing you'll need from it is README.md which has a config letting a space builder know that it's a Gradio app. Reset a remote upstream of your local rag-gradio-sample-project clone to be your freshly created Spaces repository.

The easiest way to set your space up is to set up the gradio_app folder as a git repo, set remote origin to your space repo and checkout the remote README:

```
cd gradio_app
git init
git remote add origin <your spaces repo url>
git fetch
git checkout origin/main README.md
```

The space is not working yet. You'll get the first working version after the Step 3.

- Clone https://github.com/huggingface/transformers to a local machine and run prep_scripts/markdown_to_text.py script to extract raw text from transformers/docs/source/en/. This will be your knowledge base, you don't need it to be a part of your repository

Run the command as follows (pass arguments that work for you)
```
python prep_scripts/markdown_to_text.py --input-dir transformers/docs/source/en/ --output-dir docs
```


**By design, you'll be running your experiments in a [Gradio space](https://huggingface.co/docs/hub/en/spaces-sdks-gradio). Apart from deliverables for each step you'll need to provide a link to a functioning RAG space in it final state!**

In [ ]:
https://huggingface.co/spaces/AIAlchemy1/RAG

# Steps taken to setup Gradio Space

```
git clone https://huggingface.co/spaces/AIAlchemy1/RAG

cp -r rag-gradio-sample-project/gradio_app/* RAG/

python3 -m venv env
source env/bin/activate
pip install lancedb==0.5.3 openai==1.11.1 sentence-transformers==2.3.1 tqdm==4.66.1 torch==2.1.1 transformers==4.37.2 bs4==0.0.1 markdown==3.5.1 numpy pandas pyarrow==14.0.1
pip list

git clone https://github.com/huggingface/transformers.git

python prep_scripts/markdown_to_text.py --input-dir transformers/docs/source/en/ --output-dir RAG/docs

python rag-gradio-sample-project/prep_scripts/markdown_to_text.py --input-dir transformers/docs/source/en/ --output-dir ../RAG/docs

python3 prep_scripts/lancedb_setup.py --emb-model sentence-transformers/all-MiniLM-L6-v2 --table fixed_chunks_miniLM --input-dir fixed_chunks --num-sub-vectors 64

python3 prep_scripts/lancedb_setup.py --emb-model sentence-transformers/all-MiniLM-L6-v2 --table content_aware_chunks_miniLM --input-dir content_aware_chunks --num-sub-vectors 64

python3 prep_scripts/lancedb_setup.py --emb-model BAAI/bge-large-en-v1.5 --table fixed_chunks_bge --input-dir fixed_chunks --num-sub-vectors 128

python3 prep_scripts/lancedb_setup.py --emb-model BAAI/bge-large-en-v1.5 --table content_aware_chunks_bge --input-dir content_aware_chunks --num-sub-vectors 128

pip install nltk
python -c "import nltk; nltk.download('punkt')"

python chunk_processing.py

# Setup SSH key
ssh-keygen -t ed25519 -C "se.abtahizadeh@gmail.com"
eval "$(ssh-agent -s)"
ssh-add ~/.ssh/id_ed25519
# Copy your SSH public key to the clipboard and then past into HF
cat ~/.ssh/id_ed25519.pub

git remote -v
git remote set-url origin git@hf.co:spaces/AIAlchemy1/RAG.git

app/
├── app.py
└── backend/
    ├── __init__.py
    └── query_llm.py

# Checking Environment variables
print("OPENAI_KEY:", os.getenv("OPENAI_API_KEY"))
print("HF_TOKEN:", os.getenv("HF_TOKEN"))
print("HF_MODEL:", os.getenv("HF_MODEL"))
print("TABLE_NAME:", os.getenv("TABLE_NAME"))
print("EMB_MODEL:", os.getenv("EMB_MODEL"))

git lfs install
git lfs track "*.lance"
git lfs track "*.idx"
git add .gitattributes

git add .
git commit -m "Deploy Gradio app to Hugging Face Spaces"
git push # sometimes git push --set-upstream origin master

```

### Step 1: Chunk Your Data

To efficiently pull up documents relevant to a query from a knowledge base documents are embedded and stored as vectors. Documents in your knowledge base are not expected to fit into the context length of an embedding model (most have 512 token limit). Hence chunking your documents into smaller pieces is required. Take a deeper dive into why chunking is important and what are the options [here](https://www.pinecone.io/learn/chunking-strategies/).

Your task is to implement and compare two chunking strategies: fixed-sized chunking and content-aware chunking. For content-aware you could split by sentences, paragraphs or in some other way that makes sense.

The deliverables are:
- The code for chunk splitting

In [ ]:
import os
from nltk.tokenize import sent_tokenize
import textwrap

# Ensure NLTK is set up correctly
import nltk
nltk.download('punkt')

def fixed_size_chunking(text, chunk_size=512):
    """Split text into fixed-size chunks."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def content_aware_chunking(text, max_chunk_size=512):
    """Split text by sentences to not exceed a specified maximum chunk size."""
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    for sentence in sentences:
        if len(' '.join(current_chunk + [sentence])) <= max_chunk_size:
            current_chunk.append(sentence)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
    if current_chunk:  # Add the last chunk if any
        chunks.append(' '.join(current_chunk))
    return chunks

def process_files(input_dir, fixed_output_dir, content_aware_output_dir):
    """Process files from input_dir and save chunks to output directories."""
    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            # Process with fixed-size chunking
            fixed_chunks = fixed_size_chunking(text)
            for i, chunk in enumerate(fixed_chunks):
                with open(os.path.join(fixed_output_dir, f"{filename}_chunk_{i}.txt"), 'w', encoding='utf-8') as out_file:
                    out_file.write(chunk)

            # Process with content-aware chunking
            content_chunks = content_aware_chunking(text)
            for i, chunk in enumerate(content_chunks):
                with open(os.path.join(content_aware_output_dir, f"{filename}_chunk_{i}.txt"), 'w', encoding='utf-8') as out_file:
                    out_file.write(chunk)

# Define your directories
input_dir = 'docs/'
fixed_output_dir = 'fixed_chunks/'
content_aware_output_dir = 'content_aware_chunks/'

# Ensure output directories exist
os.makedirs(fixed_output_dir, exist_ok=True)
os.makedirs(content_aware_output_dir, exist_ok=True)

# Process the documents
process_files(input_dir, fixed_output_dir, content_aware_output_dir)

### Step 2: Ingest chunks into a database and create an index

Chunks need to be vectorized and made accessible to an LLM to enable semantic search with embedding models. A current industry standard is to use a vector database to store and retrieve texts both conveniently and efficiently. There are many products out there, we'll be using [LanceDB](https://lancedb.github.io/lancedb/). LanceDB is a young product, one way it stands out is that it's embedded - it's designed not to be a standalone service but rather a part of an application, more on this [here](https://lancedb.github.io/lancedb/basic/).

Find more details on how different databases compare in [this](https://thedataquarry.com/tags/vector-db/) series of posts.

Your task is to vectorize and ingest chunked documents into the database.
**For each chunking strategy from the previous step create a separate table with one of the embedding models. Compare the chunking strategies and choose one. Perform vectorization+ingestion with the second model only with one chunking strategy of your choice**.
Use prep_scrips/lancedb_setup.py to vectorize chunks and store vector representations along with raw text in a Lancedb instance. The script also creates an index for fast ANN retrieval (not really needed for this exercise but necessary at scale). Try different embedding models and see how results differ. The options are:

- `sentence-transformers/all-MiniLM-L6-v2`: a light model, produces vectors of length 384
- `BAAI/bge-large-en-v1.5`: a much heavier model, embedding vector length is 1024

Feel free to explore other embedding models and justify your choice.
For different embedding models and different chunking strategies create different tables in the database so you can easily switch between them and compare.

Run the embedding+ingestion script as follows, make sure to look into the script and go over the arguments. Note that the number of sub-vectors for indexing must be a divisor of the model embedding size.

```
python prep_scrips/lancedb_setup.py --emb-model <model name> --table <db table name> --input-dir <folder with chunked docs> --num-sub-vectors <a number which is a divisor of the embedding dim>
```

Before committing to your space set up environment variables on the settings tab of your space, use `.env` as a ference list of all the things you can customize. Make sure to add HF_TOKEN and OPENAI_API_KEY as secrets.
Not all the parameters are required to set via environment variables, most have default values.

*The database is expected to be in the `gradio_app` folder under `.lancedb`, make sure to move it there if was initialized elsewhere.* It can be parametrized but it's unnecessary here.

To commit large files to Github use `git lfs`:
```
git lfs install
git lfs track "*.lance"
git lfs track "*.idx"
git add .gitattributes
```
Then proceed as usual.

For experimenting you can easily switch between embedding models/tables by changing the values of the corresponding env variables in your space (`EMB_MODEL`, `TABLE_NAME`). Overall, every time you change the value of an environment variable a space gets automatically rebuilt.

The deliverables are:
1. The illustration of how retrieved documents differ depending on the embedding model and the chunking strategy. You should create at least 3 tables: model_1 + chunking_strategy_1, model_1 + chunking_strategy_2, model_2 + chunking_strategy_<1 or 2>
2. The analysis of pros and cons of chunking strategies
3. The analysis of how retrieved document differ between embedding models (is one better than the other?)
4. The analysis of how the embedding time differs between models

## Example answer (fixed_chunks_MiniLM, sentence-transformers/all-MiniLM-L6-v2) without a reranker

What is Falcon model? (HuggingFace)

The Falcon model is a class of causal decoder-only languages models developed by TII. It is known for its large size, having been trained on over 1 trillion tokens, and is optimized for inference with features such as multi-query attention and efficient attention variants like FlashAttention. Falcon models are available under the Apache 2.0 license and can be found on the Hugging Face Model Hub.

What is Falcon model? (OpenAI)

Falcon is a class of causal decoder-only models developed by TII. It includes the largest Falcon checkpoints that have been trained on more than 1 trillion tokens of text, with a special focus on the RefinedWeb corpus. Falcon models are designed with a modern architecture optimized for inference, featuring multi-query attention and support for efficient attention variants like FlashAttention. The Falcon series offers both ‘base’ models, which are trained solely as causal language models, and ‘instruct’ models that have undergone additional fine-tuning. As of 2023, Falcon models are among the largest and most powerful open-source language models available, consistently achieving high rankings on the OpenLLM leaderboard. They are distributed under the Apache 2.0 license.

## Example answer (fixed_chunks_bge, BAAI/bge-large-en-v1.5) without a reranker

What is Falcon model? (HuggingFace)

Falcon is a class of causal decoder-only models built by TII. These models are designed for inference and are optimized for efficiency. They are made available under the Apache 2.0 license.

What is Falcon model? (OpenAI)

The Falcon model is a class of causal decoder-only models developed by TII. It is characterized by its training on over 1 trillion tokens of text, with a significant focus on the RefinedWeb corpus. Falcon models are designed with a modern architecture optimized for inference, featuring multi-query attention and support for efficient attention variants like FlashAttention. These models are available under the Apache 2.0 license and come in both ‘base’ versions, which are trained solely as causal language models, and ‘instruct’ versions, which have received additional fine-tuning for following instructions and conversational use. Falcon models are among the largest and most powerful open-source language models, ranking highly on the OpenLLM leaderboard.


## Example answer (content_aware_chunks_MiniLM, sentence-transformers/all-MiniLM-L6-v2) without a reranker

What is Pegasus model? (HuggingFace)

I am not familiar with the specific context provided, but based on my knowledge cutoff of 2021-09, the Pegasus model is a type of sequence-to-sequence language model proposed in the paper “Textbooks Are All You Need” by Suriya Gunasekar et al. It utilizes a variant of the encoder-decoder architecture called a fill-in-the-blank approach, where the decoder predicts missing words in a sentence based on the context provided by the previous words. The Pegasus model has been shown to achieve state-of-the-art results on several natural language processing tasks, including machine translation and text summarization.

What is Falcon model? (OpenAI)

The Pegasus model is a type of pre-trained model designed for sequence-to-sequence tasks, such as summarization. It includes several components for handling different tasks and can be used in various configurations, including PegasusModel, PegasusForConditionalGeneration, and PegasusForCausalLM for PyTorch, TFPegasusModel, and TFPegasusForConditionalGeneration for TensorFlow, and FlaxPegasusModel and FlaxPegasusForConditionalGeneration for Flax. These components allow the model to perform tasks like text generation and summarization by processing input text and producing relevant output. The model utilizes a tokenizer (PegasusTokenizer and PegasusTokenizerFast) for converting text into a format that can be processed by the model. The Pegasus model is known for its effectiveness in generating summaries and can be adapted for a wide range of text generation tasks

## Example answer (content_aware_chunks_bge, BAAI/bge-large-en-v1.5) without a reranker

What is Falcon model? (HuggingFace)

A Falcon model is a type of causal decoder-only model developed by TII that has been trained on large amounts of text data, particularly the RefinedWeb corpus. It is released under the Apache 2.0 license and is known for its modern architecture and optimization for inference, including support for efficient attention variants like FlashAttention. There are both “base” and “instruct” versions of Falcon models available, with the latter being more suitable for tasks that require following instructions or conversational use.


What is Falcon model? (OpenAI)

The Falcon model is a class of causal decoder-only models developed by TII. It includes some of the largest checkpoints trained on more than 1 trillion tokens of text, focusing on the RefinedWeb corpus. Falcon models are designed with a modern architecture optimized for inference, featuring multi-query attention and support for efficient attention variants like FlashAttention. They are available under the Apache 2.0 license and come in both ‘base’ versions, which are trained solely as causal language models, and ‘instruct’ versions, which have received additional fine-tuning for tasks requiring following instructions or conversational capabilities. As of 2023, Falcon models are among the largest and most powerful open-source language models, performing well on the OpenLLM leaderboard. They were initially available as custom code checkpoints on the Hugging Face Hub but are now fully supported in the Transformers library.


# Conclusions and deliverables

For this RAG, I have chunked the documents using following strategies
1. Chunking_strategy_1: fixed chunks
2. Chunking_strategy_2: content aware chunks

Then I used following embedding models and applied to the chunks and then loaded into the database
1. Model_1: sentence-transformers/all-MiniLM-L6-v2
2. Model_2: BAAI/bge-large-en-v1.5

Based on my observations:

1. Illustration of How Retrieved Documents Differ Chunking Strategy:

Fixed Chunks: This approach results in documents being split uniformly, regardless of their natural linguistic boundaries. This can lead to retrieved chunks that may start or end abruptly, potentially cutting off important information mid-sentence.

Content-Aware Chunks: By splitting documents based on sentences or paragraphs, this strategy should yield chunks that are more coherent and self-contained. Retrieved documents are more contextually complete, providing better insights or answers derived from a coherent piece of text.

Embedding Model:

Model_1 (sentence-transformers/all-MiniLM-L6-v2): Given its efficiency and optimization for sentence-level embeddings, this model excels in providing relevant chunks for queries that are more focused or specific, especially when used with content-aware chunking.

Model_2 (BAAI/bge-large-en-v1.5): With a larger embedding size and potentially more nuanced understanding of text, this model could be better at retrieving documents that require a deeper understanding of the query, albeit at the cost of increased computational resources.

2. Analysis of Pros and Cons of Chunking Strategies

Fixed Chunks:
Pros: Simplicity in implementation and uniformity in chunk size, which can be beneficial for certain types of analysis.
Cons: Risk of breaking sentences or thoughts, leading to less coherent chunks and potentially impacting the relevance of retrieved documents.

Content-Aware Chunks:
Pros: Maintains the integrity of the content, leading to more meaningful and coherent chunks. It improves the relevance and usefulness of retrieved documents.
Cons: Complexity in implementation and variable chunk sizes, which may introduce challenges in managing and indexing the database efficiently.

3. Analysis of How Retrieved Document Differ Between Embedding Models

Model_1 (MiniLM): Faster at embedding generation due to its smaller vector size, making it suitable for applications where speed is crucial. However, the depth of contextual understanding may be slightly less compared to larger models.

Model_2 (BAAI/bge-large-en-v1.5): Offers potentially richer and more nuanced embeddings due to its larger size, which could translate into retrieving more relevant or accurate documents for complex queries. The trade-off is higher computational demand and slower embedding times.

4. Analysis of How the Embedding Time Differs Between Models

Embedding Time: MiniLM's smaller vector size and optimized architecture should result in faster embedding times, making it a more efficient choice for real-time applications or when processing large datasets. In contrast, BAAI/bge-large-en-v1.5, with its larger vector size, will likely require more time to generate embeddings, impacting scalability and response times.

Summary:

Choosing between chunking strategies and embedding models depends on the specific requirements of the application, including the need for computational efficiency versus the demand for depth of understanding and contextual relevance. Fixed chunks and MiniLM are favored for their simplicity and speed, respectively, while content-aware chunking and BAAI/bge-large-en-v1.5 might be preferred for applications requiring higher coherence and deeper textual understanding.

In [ ]:
# Embed documents with different chunking strategies and ingest into the database

### Step 3: Add a reranker

A reranker is a second-level model which produces similarity scores for pairs of (input query + retrieved document). Cross-encoders are conventionally used for reranking, their architecture is slightly different from retrieval models (more on it [here] and [here](https://www.sbert.net/examples/applications/retrieve_rerank/README.html)). Cross-encoders are much more costly to run, therefore a retrieval model is used to get a few (dozens) highest-scoring items, and a reranker picks the best among these. The overall pipeline is similar to the recommender system industry standard: a light model retrieves top-n, a precise and heavy model reranks n to get top k, n is orders of magnitude larger than k.

Cross-encoders are optional because of the overhead their usage implies. Your task is to implement a reranker using a cross-encoder and assess pros and cons of having it. Do not forget that the process of pulling the most relevant documents becomes two-staged: retrieve a larger number of items first, than rerank and keep the best top-k for context.

The models fit for the task:
1. BAAI/bge-reranker-large
2. cross-encoder/ms-marco-MiniLM-L-6-v2

As usual, feel free to pick another model and provide some description to it.

The deliverables are:

1. The code that enables a reranker.
3. A comparison of how the prompt and the model output change after adding a reranker
4. The analysis of pros and cons. The evaluation aspects should include the relevance of the top-k documents, the response time.


# Steps taked to add a reranker

1. We initialize the cross-encoder/ms-marco-MiniLM-L-6-v2 model. This model is designed to take pairs of texts (e.g., a query and a document) and output a relevance score indicating how well the document matches the query.

2. We create pairs of the input query with each of the top-N retrieved documents.

3. The model predicts the relevance scores for each query-document pair. These scores indicate the relevancy of each document to the query.

4. The documents are sorted based on their scores, and the top-K documents are selected as the final output.

In [ ]:
# reranker.py
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

class Reranker:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def rerank(self, query, documents):
        # Pair the query with each document and encode them
        pairs = [[query, doc] for doc in documents]
        encoded_pairs = self.tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)

        # Predict relevance scores using the cross-encoder model
        with torch.no_grad():
            scores = self.model(**encoded_pairs).logits[:,0]  # Assuming binary classification: [not relevant, relevant]

        # Sort documents by their scores in descending order
        ranked_docs = [doc for _, doc in sorted(zip(scores, documents), key=lambda x: x[0], reverse=True)]

        return ranked_docs

In [ ]:
# semantic_search.py after adding the reranker
import lancedb
import os
import gradio as gr
from sentence_transformers import SentenceTransformer

# Reranker class is defined in the Reranker.py
from reranker import Reranker

db = lancedb.connect(".lancedb")

TABLE = db.open_table(os.getenv("TABLE_NAME"))
VECTOR_COLUMN = os.getenv("VECTOR_COLUMN", "vector")
TEXT_COLUMN = os.getenv("TEXT_COLUMN", "text")
BATCH_SIZE = int(os.getenv("BATCH_SIZE", 32))

retriever = SentenceTransformer(os.getenv("EMB_MODEL"))

reranker_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = Reranker(reranker_model_name)  # Initialize the reranker

def retrieve(query, k):
    query_vec = retriever.encode(query)
    try:
        documents = TABLE.search(query_vec, vector_column_name=VECTOR_COLUMN).limit(k).to_list()
        documents_text = [doc[TEXT_COLUMN] for doc in documents]

        # Rerank the retrieved documents using the cross-encoder model
        reranked_documents = reranker.rerank(query, documents_text)

        return reranked_documents

    except Exception as e:
        raise gr.Error(str(e))

## Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2)

What is Falcon model? (HuggingFace)

A Falcon model is a large and powerful open-source language model developed by TII, known for its modern and optimized architecture for inference. It is built as a causal decoder-only model and has been trained on a vast amount of text data, particularly emphasizing the RefinedWeb corpus. Falcon models are now fully supported in the Transformers library and come under the Apache 2.0 license.

What is Falcon model? (OpenAI)

The Falcon model is a class of causal decoder-only models built by TII. It is notable for being trained on more than 1 trillion tokens of text, with a significant focus on the RefinedWeb corpus. Falcon models are available under the Apache 2.0 license and feature a modern architecture optimized for inference. This includes multi-query attention and support for efficient attention variants like FlashAttention. Falcon models come in both ‘base’ versions, which are trained only as causal language models, and ‘instruct’ versions, which have received further fine-tuning.

# Conclusions and deliverables

Based on the example prompt and answers provided, both with and without the reranker, here are some observations and analysis regarding the impact of adding a reranker to the model output, as well as pros and cons:

1. The answer with the reranker seems to provide more specific details, such as mentioning the "RefinedWeb corpus" and explicitly stating the model's support in the Transformers library. This suggests that the reranker might be selecting documents that offer more detailed and possibly more relevant information to the query.

2. The reranked response focuses on the model's architecture and its licensing, whereas the answer without the reranker emphasizes the model's training data size and specific technical optimizations. This indicates that the reranker can influence the aspect of the topic that is highlighted in the response, potentially aligning better with user intent or providing a broader perspective.

2. Analysis of Pros and Cons

Pros:

1. The reranker appears to enhance the relevance of the provided information by selecting chunks or generating responses that are more closely aligned with the query's context. This can significantly improve user satisfaction by providing answers that meet their informational needs more directly.

2. By prioritizing responses that contain specific details and comprehensive insights, the reranker can improve the overall quality of the content presented to the user.

Cons:

1. The reranking process introduces an additional computational step, which can impact the response time (in this case about 1 second more). Depending on the complexity of the reranker model and the amount of data being processed, this could lead to noticeable delays in providing answers, affecting the user experience in time-sensitive applications.

2. Utilizing a reranker, especially a sophisticated model like a cross-encoder, increases computational resource consumption. This can be a significant factor when operating at scale or in environments with limited computing capacity.

Overall, adding a reranker can significantly enhance the quality and relevance of model outputs, making it a valuable tool for refining search results or generated content. However, the benefits in content relevance and quality must be balanced against the increased computational demands and potential impact on response times. In practice, the decision to use a reranker should consider the specific requirements and constraints of the application, including the importance of response speed and available computational resources.

In the context of examples above, the reranker seems to effectively adjust the focus and detail level of the responses, suggesting it is successfully identifying and prioritizing more relevant or comprehensive content. This indicates that, for applications where depth and specificity of information are critical, the benefits of reranking may outweigh the cons related to performance and resource usage.

### Step 4: Try a different LLM

The suggested `Mistral-7b-instruct` is a great but small model for an LLM. A larger model can be applied to a wider range of problems and do more complex reasoning. Within the scope of this project a larger model may not be beneficial but for more complex cases the difference would become apparent. Another dimension to explore is a base model which was not instruction fine-tuned - it won't respond to your queries the way you'd expect. It may be a great exercise to see the value of fine-tuning.

The task here is to try out an alternative LLM to explore the differences.

The options are:
1. mistralai/Mistral-7B-v0.1
2. mistralai/Mixtral-8x7B-Instruct-v0.1

Of course, feel free to choose another one and give some details on how different it is from the initial model.

The deliverables are:

1. The comparison between outputs of the Mistral-7b-instuct and a different model of your choice.
2. The difference in response times if a larger model was chosen. Make sure to make multiple queries to make the comparison meaningful.
3. Analyse the differences between outputs and share the conclusions.


### Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2) and using a larger LLM (mistralai/Mixtral-8x7B-Instruct-v0.1)

What is Falcon model? (HuggingFace)

The Falcon model is a class of causal decoder-only models built by TII (Telecommunication Intelligence Industry), which are available under the Apache 2.0 license. These models have been trained on over 1 trillion tokens of text data, particularly focusing on the RefinedWeb corpus. Falcon models utilize a modern architecture optimized for inference, featuring multi-query attention and support for efficient attention variants such as FlashAttention. Initially introduced as custom code checkpoints, Falcon models are now fully integrated into the Hugging Face Transformers library.

What is Falcon model? (OpenAI)

Falcon is a class of causal decoder-only models built by TII. The largest Falcon checkpoints have been trained on >=1T tokens of text, with a particular emphasis on the RefinedWeb corpus. They are made available under the Apache 2.0 license. Falcon’s architecture is modern and optimized for inference, featuring multi-query attention and support for efficient attention variants like FlashAttention. The models include both ‘base’ models trained only as causal language models and ‘instruct’ models that have received further fine-tuning.

# Conclusions and deliverables

1. Comparison between Outputs of Mistral-7b-Instruct and Mixtral-8x7B-Instruct-v0.1

Both models provide a comprehensive overview of the Falcon model, including its development by TII, architecture, licensing, and corpus focus. However, the Mixtral-8x7B-Instruct-v0.1 descriptions seem to offer a slightly more detailed explanation, particularly in specifying the Telecommunication Intelligence Industry (TII) and emphasizing the model's optimization for inference. This suggests that the larger model might have a slightly better capability to pull and organize detailed information.

The outputs are consistent across both models in terms of accuracy, mentioning key features like the causal decoder-only architecture, Apache 2.0 license, and the RefinedWeb corpus. This consistency indicates that both models have a reliable understanding of the Falcon model and can accurately reflect its main characteristics.

The Mixtral-8x7B-Instruct-v0.1 model's responses include nuanced details such as the integration into the Hugging Face Transformers library and the distinction between 'base' and 'instruct' models. This suggests that larger models may be better at capturing and conveying nuanced differences within a topic.

B. Difference in Response Times with a Larger Model

Using a larger model like the Mixtral-8x7B-Instruct-v0.1 generally leads to longer response times (in this case about 1 second more) compared to smaller models such as the Mistral-7b-Instruct. This is because larger models require more computational resources to process the same input. The complexity and size of the model directly impact how quickly it can generate responses. Given that larger models may generate more detailed and nuanced outputs, the additional time could also be attributed to the complexity of the processing involved in producing such responses. There's often a trade-off between response efficiency and the depth or quality of the output. While larger models might take longer to produce responses, the results are potentially more comprehensive and detailed.

C. Analysis of Differences and Conclusions

The main differences between the outputs of the Mistral-7b-Instruct and Mixtral-8x7B-Instruct-v0.1 models are in the level of detail and specificity provided in their responses. While both models accurately describe the Falcon model, the larger Mixtral-8x7B-Instruct-v0.1 seems to offer slightly more detailed and nuanced information.

Conclusions:

The larger Mixtral-8x7B-Instruct-v0.1 model appears to utilize its increased capacity to provide responses with more depth and specificity. This suggests that larger models might be preferred for applications requiring detailed explanations or nuanced understanding of complex topics. When choosing between models, there's a trade-off between computational efficiency and the depth of response. Larger models may offer more detailed information but at the cost of longer processing times. The choice depends on the specific requirements of the task, such as the need for depth and detail versus the importance of quick responses. The comparison suggests that for tasks requiring high accuracy and detail, larger models like Mixtral-8x7B-Instruct-v0.1 might be more suitable, despite potential increases in response time. For applications where speed is critical, smaller models may be preferable, provided they meet the accuracy and detail requirements of the task.

In [ ]:
# Analysis of the difference between LLMs

### Step 5 (Bonus): Use an LLM to quantitatively compare outputs of different variants of the system (LLM as a Judge)

Use a powerful LLM (e.g. GPT-4) to quantitatively evaluate outputs of two alternative setups (different embedding models, different LLMs, both etc.). For inspiration and for prompts refer to [1](https://arxiv.org/pdf/2306.05685.pdf), [2](https://arxiv.org/pdf/2401.10020.pdf), [3](https://www.airtrain.ai/blog/the-comprehensive-guide-to-llm-evaluation#high-level-approach)

The deliverables:

1. The code you put together
2. The high-level description of the setup
3. The results of the qualitative comparison


In [ ]:
def evaluate_response(prompt, response):
    evaluation_prompt = f"Given the prompt: '{prompt}', and the response: '{response}', rate the response's relevance, coherence, and informativeness on a scale from 1 to 10."
    evaluation = openai.Completion.create(
        engine="gpt-4-turbo-preview",
        prompt=evaluation_prompt,
        max_tokens=100,
        temperature=0.7,
    )
    return evaluation.choices[0].text.strip()

In [ ]:
prompt = ["What is the Falcon model?"]
# Example answer without a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2)
response_1 = ["The Falcon model is a class of causal decoder-only languages models developed by TII. It is known for its large size, having been trained on over 1 trillion tokens, and is optimized for inference with features such as multi-query attention and efficient attention variants like FlashAttention. Falcon models are available under the Apache 2.0 license and can be found on the Hugging Face Model Hub."]
# Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2)
response_2 = ["A Falcon model is a large and powerful open-source language model developed by TII, known for its modern and optimized architecture for inference. It is built as a causal decoder-only model and has been trained on a vast amount of text data, particularly emphasizing the RefinedWeb corpus. Falcon models are now fully supported in the Transformers library and come under the Apache 2.0 license."]
# Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2) and using a larger LLM (mistralai/Mixtral-8x7B-Instruct-v0.1)
response_3 = ["The Falcon model is a class of causal decoder-only models built by TII (Telecommunication Intelligence Industry), which are available under the Apache 2.0 license. These models have been trained on over 1 trillion tokens of text data, particularly focusing on the RefinedWeb corpus. Falcon models utilize a modern architecture optimized for inference, featuring multi-query attention and support for efficient attention variants such as FlashAttention. Initially introduced as custom code checkpoints, Falcon models are now fully integrated into the Hugging Face Transformers library."]

evaluation_1 = evaluate_response(prompt, response_1)
evaluation_2 = evaluate_response(prompt, response_2)
evaluation_3 = evaluate_response(prompt, response_1)

print(f"Response 1: {evaluation_1}\n")
print(f"Response 2: {evaluation_2}\n")
print(f"Response 3: {evaluation_3}\n")


- Response 1 - Example answer without a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2)

Relevance: 10/10. This response directly addresses the question by describing the Falcon model as a class of causal decoder-only language models developed by TII, mentioning its training on over 1 trillion tokens, features, and licensing.

Coherence: 9/10. The response is well-structured and coherent, with a logical flow of information from the model's description to its features and availability. The only minor deduction is due to the slight ambiguity about the specific applications or impact of these features.

Informativeness: 9/10. It provides a comprehensive overview, including the model's size, optimization features, licensing, and availability. It could be slightly improved by adding examples of applications or the significance of the mentioned features to further contextualize its capabilities.

- Response 2 - Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2)

Relevance: 10/10. It effectively answers the prompt by detailing the Falcon model's development by TII, its architecture, training data emphasis, support in the Transformers library, and licensing.

Coherence: 10/10. This response is exceptionally coherent, presenting a clear and concise overview of the Falcon model, its development, and key characteristics in a well-organized manner.

Informativeness: 8/10. While it covers essential aspects of the Falcon model, including its open-source nature and focus on the RefinedWeb corpus, it lacks the explicit mention of specific features like multi-query attention or FlashAttention, slightly limiting its informativeness.

- Response 3 - Example answer with a reranker (fixed_chunks_miniLM, sentence-transformers/all-MiniLM-L6-v2) and using a larger LLM (mistralai/Mixtral-8x7B-Instruct-v0.1)

Relevance: 10/10. Directly relevant to the prompt, this response expands on the Falcon model's development, training data, architecture, and integration into the Hugging Face Transformers library.

Coherence: 10/10. The response is coherent and well-structured, smoothly transitioning from the model's basic description to more detailed aspects like its architecture and library integration.

Informativeness: 10/10. This response is highly informative, providing details on the model's training, specific architectural features, initial introduction, and full support in the Transformers library. It strikes an excellent balance between breadth and depth of information.